In [1]:
# Dependencies
import tweepy
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import style
import time
style.use('ggplot')

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Import API Keys
from config import (consumer_key, consumer_secret, 
                    access_token, access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [2]:
# Target Term for twitter bot account
target_term = "@CWBot2"

# Initialize user list
target_user_list = []

In [3]:
# Define function
def SentiAnalysisReq():
    
    # Variables for holding sentiments
    sentiments = []

    # Variable for max_id
    oldest_tweet = None

    # Counter
    counter = 1
    
    # Search for tweets mentioning bot account
    public_tweets = api.search(target_term, count=500, result_type="recent")
    
    # Loop through searched tweets
    for tweet in public_tweets["statuses"]:
        
        # Pull tweet author 
        tweet_author = tweet["user"]["screen_name"]
        
        # Pull account mentioned 
        target_user = tweet["entities"]["user_mentions"][1]["screen_name"]
        
        # Initialize if/else to eliminate accounts already analyzed
        if (target_user in target_user_list):
            print("This account has been previously analyzed")
        
        else:
            
            # Add account to target_user_list
            target_user_list.append(target_user)
            print(target_user_list)

            # Loop through pages of tweets for sentitment analysis 
            for x in range(1, 26):
                public_tweets = api.user_timeline(target_user, page=x)

                # Loop through tweets
                for tweet in public_tweets:
                    
                    # Get sentiment results for each tweet
                    results = analyzer.polarity_scores(tweet["text"])
                    compound = results["compound"]
                    pos = results["pos"]
                    neu = results["neu"]
                    neg = results["neg"]
                    
                    # Add to data to sentiments dictionary 
                    sentiments.append({"Date": tweet["created_at"],"Compound": compound, "Positive": pos,"Negative": neu, "Neutral": neg,"Tweets Ago": counter})
                    
                    # Add to counter
                    counter += 1

            # Convert sentiments to data frame
            sentiments_pd = pd.DataFrame.from_dict(sentiments)
            
            # Create chart
            x_vals = sentiments_pd["Tweets Ago"]
            y_vals = sentiments_pd["Compound"]
            plt.plot(x_vals, y_vals, marker="o", linewidth=0.5, alpha=0.8)
            now = datetime.now()
            now = now.strftime("%m/%d/%Y")
            plt.title(f"Sentiment Analysis of Tweets ({now}) for {target_user}")
            plt.xlim([x_vals.max(),x_vals.min()])
            plt.ylabel("Tweet Polarity")
            plt.xlabel("Tweets Ago")

            # Save chart 
            plt.savefig(f"SentimentAnalysis{target_user}.png")
            
            #Plot is cleaned for the next analysis
            plt.clf()
            plt.cla()
            plt.close()
            
            # Tweet chart
            api.update_with_media(f"SentimentAnalysis{target_user}.png",f"New Tweet Analysis: {target_user} (Thanks @{tweet_author}!!)")

In [ ]:
# Run function at 5 minute intervals
while(True):
        SentiAnalysisReq()
        time.sleep(300)

['capgaznews']
['capgaznews', 'capitalgazette']
['capgaznews', 'capitalgazette', 'baltimoresun']
['capgaznews', 'capitalgazette', 'baltimoresun', 'washingtonpost']
['capgaznews', 'capitalgazette', 'baltimoresun', 'washingtonpost', 'TheYoungTurks']
['capgaznews', 'capitalgazette', 'baltimoresun', 'washingtonpost', 'TheYoungTurks', 'weatherchannel']
This account has been previously analyzed
This account has been previously analyzed
This account has been previously analyzed
This account has been previously analyzed
This account has been previously analyzed
This account has been previously analyzed
